# Financial Report Generator with DeepSeek Workflow

This notebook demonstrates a simple asynchronous financial reporting system that leverages a GANET-style workflow with the `deepseek-r1:14b` model (accessed via the `ollama` package). The code is organized into three main agents:

- **Data Collection Agent:** Prompts DeepSeek to generate raw financial data as a JSON array.
- **Analysis Agent:** Validates and aggregates the financial data (income, expenses, and net profit) using Pydantic models.
- **Presentation Agent:** Formats the computed report into a Markdown presentation for clear, actionable insights.

Together, these components illustrate how to integrate advanced natural language processing with structured data validation and asynchronous programming for dynamic financial reporting.


In [4]:
import asyncio
import json
import re
from pydantic import BaseModel, ValidationError
from typing import List
from ollama import AsyncClient

def extract_json(text: str) -> str:
    """
    Extracts the first JSON array found in the text.
    Raises ValueError if no valid JSON array is found.
    """
    match = re.search(r'\[.*\]', text, re.DOTALL)
    if match:
        return match.group(0)
    raise ValueError("No JSON array found")

class FinancialItem(BaseModel):
    description: str
    amount: float
    category: str

class FinancialReport(BaseModel):
    report_name: str
    date: str
    items: List[FinancialItem]
    total_income: float
    total_expenses: float
    net_profit: float

#an agent to collect data
class DataCollectionAgent:
    async def collect_data(self, client: AsyncClient, report_name: str, date: str) -> List[FinancialItem]:
        prompt = (
            f"Generate a JSON array for the financial report '{report_name}' on {date}. "
            "Each element in the array must be an object with exactly the following keys: "
            "'description' (a string), 'amount' (a number), and 'category' (a string, either 'income' or 'expense'). "
            "Return ONLY the JSON output without any additional commentary or formatting. "
            "For example:\n"
            "[\n"
            "  {\"description\": \"Income from sales\", \"amount\": 10000, \"category\": \"income\"},\n"
            "  {\"description\": \"Rent expense\", \"amount\": 2000, \"category\": \"expense\"}\n"
            "]"
        )
        
        response = await client.chat(
            #shows its chain-of-thought before executing workflow
            model='deepseek-r1:14b',
            messages=[{'role': 'user', 'content': prompt}],
            #low temp for consistency
            options={'temperature': 0.2}, 
        )
        
        raw_content = response.message.content.strip()
        print("DEBUG: Raw response from model:")
        print(raw_content)
        
        if not raw_content:
            raise ValueError("Empty response from the model")
        
        try:
            json_text = extract_json(raw_content)
            response_json = json.loads(json_text)
            data = [FinancialItem.model_validate(item) for item in response_json]
            return data
        except json.JSONDecodeError as e:
            print(f"Invalid JSON response: {e}\nContent: {raw_content}")
            raise ValueError("Invalid JSON response from the model") from e
        except ValidationError as e:
            print(f"Validation error: {e}")
            raise
        except Exception as e:
            print(f"An error occurred in data collection: {e}")
            raise

#an agent to do an analysis on financial data
class AnalysisAgent:
    async def analyze_data(self, data: List[FinancialItem]) -> FinancialReport:
        total_income = sum(item.amount for item in data if item.category == 'income')
        total_expenses = sum(item.amount for item in data if item.category == 'expense')
        net_profit = total_income - total_expenses
        
        report = FinancialReport(
            report_name="Monthly Financial Report",
            date="2025-02-21",
            items=data,
            total_income=total_income,
            total_expenses=total_expenses,
            net_profit=net_profit
        )
        return report

#an agent to present the data in clean, readbale format
class PresentationAgent:
    async def create_presentation(self, client: AsyncClient, report: FinancialReport) -> str:

        prompt = (
            f"Based on the following financial report details, create a Markdown formatted presentation.\n\n"
            f"Report Name: {report.report_name}\n"
            f"Date: {report.date}\n"
            f"Total Income: {report.total_income}\n"
            f"Total Expenses: {report.total_expenses}\n"
            f"Net Profit: {report.net_profit}\n\n"
            f"Items:\n{json.dumps([item.model_dump() for item in report.items], indent=2)}\n\n"
            "The presentation should include:\n"
            "- A summary section clearly stating the total income, expenses, and net profit (using the numbers above).\n"
            "- A detailed breakdown section listing each item with its description, amount, and category.\n"
            "- A recommendations section offering actionable insights based on the financial data.\n\n"
            "Ensure the output is complete, with no placeholder text, and ONLY include the Markdown formatted presentation."
        )
        
        try:
            response = await client.chat(
                model='deepseek-r1:14b',
                messages=[{'role': 'user', 'content': prompt}],
                options={'temperature': 0.2},
            )
            
            presentation = response.message.content.strip()
            return presentation
        except Exception as e:
            print(f"An error occurred in presentation generation: {e}")
            raise

async def main():
    client = AsyncClient()
    
    report_name = "Monthly Financial Report"
    date = "2025-02-21"
    
    try:
        collection_agent = DataCollectionAgent()
        data = await collection_agent.collect_data(client, report_name, date)
        analysis_agent = AnalysisAgent()
        report = await analysis_agent.analyze_data(data)
        presentation_agent = PresentationAgent()
        presentation = await presentation_agent.create_presentation(client, report)
        
        print("Generated Financial Report:")
        print(report.model_dump_json(indent=2))
        print("\nPresentation:")
        print(presentation)
    except Exception as e:
        print(f"Failed to generate financial report: {e}")

if __name__ == '__main__':
    await main()

DEBUG: Raw response from model:
<think>
Okay, so the user wants me to generate a JSON array for a monthly financial report on February 21, 2025. Each object in the array needs to have 'description', 'amount', and 'category' as keys. The category should be either 'income' or 'expense'. They also provided an example, which is helpful.

First, I need to figure out what kind of financial transactions are typical for a monthly report. Probably a mix of incomes and expenses. Let me think about common categories like salary, rent, utilities, etc.

I'll start by listing some income sources. Maybe 'Salary Income' as the main one. Then, perhaps some investment returns or other income streams. For expenses, essential ones like rent, utilities, groceries, transportation, insurance, and maybe a few others like internet or dining out.

Next, I need to assign amounts to each of these. The example had 10000 for income and 2000 for expense. I should vary the amounts to make it realistic but keep them r